In [68]:
import sys

import datetime
import pickle

import numpy as np
import pandas as pd
import operator
import random
from  scipy import stats



from scipy.stats import norm
from sklearn.neighbors import KernelDensity



import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
import plotly.figure_factory as ff


########## to be able to plot offline (without sending the plots to the plotly server every time)
import plotly

import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
################



from IPython.core.display import display,HTML
display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser

In [69]:

df_states_codes =pd.read_csv('../Data/Mine/US_states_codes.csv')

df_states_codes

,State,State_code
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA
5,Colorado,CO
6,Connecticut,CT
7,Delaware,DE
8,District of Columbia,DC
9,Florida,FL


In [70]:

df_crime_tot =pd.read_csv('../Data/Mine/Crime_TOTAL_US_1960_2014_Added_per100000.csv')

df_crime_tot



,Year,Population,Violent crime total,Murder and nonnegligent Manslaughter,Murder_per100000,Legacy rape /1,Revised rape /2,Robbery,Aggravated assault
0,1960,179323175,288460,9110,5.080213,17190,NaN,107840,154320
1,1961,182992000,289390,8740,4.776165,17220,NaN,106670,156760
2,1962,185771000,301510,8530,4.591675,17550,NaN,110860,164570
3,1963,188483000,316970,8640,4.583968,17650,NaN,116470,174210
4,1964,191141000,364220,9360,4.896909,21420,NaN,130390,203050
5,1965,193526000,387390,9960,5.146595,23410,NaN,138690,215330
6,1966,195576000,430180,11040,5.644864,25820,NaN,157990,235330
7,1967,197457000,499930,12240,6.198818,27620,NaN,202910,257160
8,1968,199399000,595010,13800,6.920797,31670,NaN,262840,286700
9,1969,201385000,661870,14760,7.329245,37170,NaN,298850,311090


In [71]:

df_crime_state_by_state =pd.read_csv('../Data/Mine/CrimeStatebyState_1960-2014_added_per100000.csv')

df_crime_state_by_state

,State,Year,Population,Violent_crime_total,Murder_and_nonnegligent_Manslaughter,Murder_per100000,Legacy_rape_/1,Revised_rape_/2,Robbery,Aggravated_assault
0,Alabama,1960,3266740,6097,406,12.428292,281,NaN,898,4512
1,Alabama,1961,3302000,5564,427,12.931557,252,NaN,630,4255
2,Alabama,1962,3358000,5283,316,9.410363,218,NaN,754,3995
3,Alabama,1963,3347000,6115,340,10.158351,192,NaN,828,4755
4,Alabama,1964,3407000,7260,316,9.275022,397,NaN,992,5555
5,Alabama,1965,3462000,6916,395,11.409590,367,NaN,992,5162
6,Alabama,1966,3517000,8098,384,10.918396,341,NaN,1124,6249
7,Alabama,1967,3540000,8448,415,11.723164,371,NaN,1167,6495
8,Alabama,1968,3566000,8288,421,11.805945,396,NaN,1462,6009
9,Alabama,1969,3531000,8842,485,13.735486,494,NaN,1448,6415


In [72]:
df_merged = pd.merge(df_crime_state_by_state, df_states_codes, on='State', how='left')

df_merged

,State,Year,Population,Violent_crime_total,Murder_and_nonnegligent_Manslaughter,Murder_per100000,Legacy_rape_/1,Revised_rape_/2,Robbery,Aggravated_assault,State_code
0,Alabama,1960,3266740,6097,406,12.428292,281,NaN,898,4512,AL
1,Alabama,1961,3302000,5564,427,12.931557,252,NaN,630,4255,AL
2,Alabama,1962,3358000,5283,316,9.410363,218,NaN,754,3995,AL
3,Alabama,1963,3347000,6115,340,10.158351,192,NaN,828,4755,AL
4,Alabama,1964,3407000,7260,316,9.275022,397,NaN,992,5555,AL
5,Alabama,1965,3462000,6916,395,11.409590,367,NaN,992,5162,AL
6,Alabama,1966,3517000,8098,384,10.918396,341,NaN,1124,6249,AL
7,Alabama,1967,3540000,8448,415,11.723164,371,NaN,1167,6495,AL
8,Alabama,1968,3566000,8288,421,11.805945,396,NaN,1462,6009,AL
9,Alabama,1969,3531000,8842,485,13.735486,494,NaN,1448,6415,AL


In [ ]:
df_merged.to_csv(path_or_buf='../Data/Mine/CrimeStatebyState_1960-2014_added_per100000_abr.csv', sep=',', na_rep='NA')
print ("written: ../Data/Mine/CrimeStatebyState_1960-2014_added_per100000_abr.csv")

In [ ]:
df_sected_crime = df_crime_state_by_state[df_crime_state_by_state['State']== 'Alabama' ]

df_sected_crime

In [ ]:
for state in df_crime_state_by_state.State.unique():
    print (state)

In [ ]:
# Create and style traces

list_interesting_states = ['Alabama', 'California', 'Illinois', 'Massachusetts', 'Missouri', 'New York', 'Tennessee', 'Wyoming']

data = []

trace0 = go.Scatter(
    x = df_crime_tot['Year'],
    y = df_crime_tot['Murder_per100000'],
    name = 'tot. US crime',
    line = dict(
        color = ('rgb(0, 0, 0)'),
        width = 4)
)
data.append(trace0)

## i remove DC for now, since its murder rate is one order of magnitude higher than anywhere else
df_sected_crime_no_DC = df_crime_state_by_state[df_crime_state_by_state['State']!= 'District of Columbia' ]


#for state in df_crime_state_by_state.State.unique():
for state in list_interesting_states:   
    
    
    
    df_sected_crime = df_sected_crime_no_DC[df_sected_crime_no_DC['State']== state ]
    
    
    trace1 = go.Scatter(
        x = df_sected_crime['Year'],
        y = df_sected_crime['Murder_per100000'],
        name = state,

    )
    data.append(trace1)

    
# Edit the layout
layout = dict(
#     title = 'Average High and Low Temperatures in New York',
              xaxis = dict(title = 'Year'),
              yaxis = dict(title = 'Average number of murders per 100,000 people'),
              )

fig = dict(data=data, layout=layout)


plotly.offline.iplot(fig)


offline.plot(fig, auto_open=True, image = 'png', image_filename="time_evol_tot_crime" ,image_width=2000, image_height=1000, 
              filename='/home/staff/julia/at_Northwestern/US_Crime/Figures/time_evol_tot_crime.html', validate=True)




In [ ]:
offline.plot(fig, auto_open=True, image = 'png', image_filename="map_us_crime_"+str(year),
             image_width=2000, image_height=1000, 
             filename='/your_path/'+"map_us_crime_"+str(year)+'.html', validate=True)


In [ ]:



year = 1960

df_sected_crime = df_merged[df_merged['State']!= 'District of Columbia' ]

df_sected_crime = df_merged[df_merged['Year']== year ]
  

for col in df_sected_crime.columns:
    df_sected_crime[col] = df_sected_crime[col].astype(str)
    

# scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\   # purple
#             [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]


scl = [[0.0, '#ffffff'],[0.2, '#ff9999'],[0.4, '#ff4d4d'], \
       [0.6, '#ff1a1a'],[0.8, '#cc0000'],[1.0, '#4d0000']] # reds



# #ffe6e6
# 90% 		 #ffcccc
# 85% 		 #ffb3b3
# 80% 		 #ff9999
# 75% 		 #ff8080
# 70% 		 #ff6666
# 65% 		 #ff4d4d
# 60% 		 #ff3333
# 55% 		 #ff1a1a
# 50% 	

#  #ff0000
# 45% 		 #e60000
# 40% 		 #cc0000
# 35% 		 #b30000
# 30% 		 #990000
# 25% 		 #800000
# 20% 		 #660000
# 15% 		 #4d0000



df_sected_crime['text'] = df_sected_crime['State'] + '<br>' +\
    'Pop: ' + df_sected_crime['Population'] + '<br>' +\
    'Tot. Violent crimes: '+df_sected_crime['Violent_crime_total'] + '<br>' +\
    'Murder rate: '+df_sected_crime['Murder_per100000']

data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = df_sected_crime['State_code'],
        z = df_sected_crime['Murder_per100000'].astype(float),
        locationmode = 'USA-states',
        text = df_sected_crime['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',  # while lines to separate states
                width = 2
            ) ),
        colorbar = dict(
            title = "Murder rate per 100,000 people",
           # tickvals= [1,5,10,20],
            #ticktext= [1,5,10,20],
           # colorscale = scl,
#             cmin=1,
#             cmax=20,
            #color = df_upgrading.month
            )
        ) ]




layout = dict(
        title = year,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
#              showlakes = True,
#             lakecolor = 'rgb(73, 216, 230)'  
            ),
             )
    
fig = dict( data=data, layout=layout )

plotly.offline.iplot(fig)



offline.plot(fig, auto_open=True, image = 'png', image_filename="map_us_crime_"+str(year) ,image_width=2000, image_height=1000, 
              filename='/home/staff/julia/at_Northwestern/US_Crime/Figures/'+"map_us_crime_"+str(year)+'.html', validate=True)



In [ ]:
## US crime map + slider:

### colorscale    
# scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\  # purples
#             [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]



scl = [[0.0, '#ffffff'],[0.2, '#ff9999'],[0.4, '#ff4d4d'], [0.6, '#ff1a1a'],[0.8, '#cc0000'],[1.0, '#660000']]  # reds

    
data_slider = []


#### I create a list of data_dicts, one per year that will be displayed with the slider
for year in df_merged.Year.unique():
    print (year)
 
   # df_sected_crime = df_merged[df_merged['State']!= 'District of Columbia' ]
    df_sected_crime = df_merged[df_merged['Year']== year ]
    
    
    for col in df_sected_crime.columns:
        df_sected_crime[col] = df_sected_crime[col].astype(str)

    df_sected_crime['text'] = df_sected_crime['State'] + '<br>' +\
    'Pop: ' + df_sected_crime['Population'] + '<br>' +\
    'Tot. Violent crimes: '+df_sected_crime['Violent_crime_total'] + '<br>' +\
    'Murder rate: '+df_sected_crime['Murder_per100000']

  
    data_one_year = dict(
                        type='choropleth',
                        locations = df_sected_crime['State_code'],
                        z=df_sected_crime['Murder_per100000'].astype(float),
                        locationmode='USA-states',
                        colorscale = scl,
                        text = df_sected_crime['text'],
                        colorbar = dict(
                            title = "Murder rate per 100,000 people")
                        )

    data_slider.append(data_one_year)  # I add the dictionary to the list of dictionaries for the slider
  
    
    
    

# let's create the steps for the slider
steps = []
for i in range(len(data_slider)):
    step = dict(method='restyle',
                args=['visible', [False] * len(data_slider)],
                label='Year {}'.format(i + 1960))
    step['args'][1][i] = True
    steps.append(step)

sliders = [dict(active=0,
                pad={"t": 1},
                steps=steps)]  



layout = dict(geo=dict(scope='usa',
                       projection={'type': 'albers usa'}),
              sliders=sliders)


fig = dict(data=data_slider, layout=layout)




plotly.offline.iplot(fig)


offline.plot(fig, auto_open=True, image = 'png', image_filename="map_us_crime_slider" ,image_width=2000, image_height=1000, 
              filename='/home/staff/julia/at_Northwestern/US_Crime/Figures/map_us_crime_slider.html', validate=True)


In [ ]:
## NOTE:  converter of code into html:  
##  http://hilite.me/

## if the background of the blog is black, use either 'native' or 'tango' as styles in the drop-down menu

In [ ]:
#sorted(df_crime_cities['1985'].unique())

In [ ]:
df_crime_cities

In [73]:
###  crime map - CITIES  (fixed color)

df_crime_cities = pd.read_csv('../Data/Mine/LocalCrimeTrendsInOneVar_crime_rate_vs_time_cities_500000plus_1985_2014.csv')


# i need the all-time maximum value to rescale the city circles accordingly
max_value_years = 0
i = 1985
while i <=2014:   
    value = df_crime_cities[str(i)].max()
    if  value > max_value_years:
        max_value_years = value
    i  +=1





cities = []  # data structure for the plot (list of dict)


year = 1985

df_for_plotting = df_crime_cities[[str(year),'lon','lat','City']]
df_for_plotting.dropna(subset=[str(year)], inplace=True)

df_for_plotting['text'] = df_for_plotting['City'] + '<br>Crime rate: ' + (df_for_plotting[str(year)]).astype(str)


city = dict(
    type = 'scattergeo',
    locationmode = 'USA-states',
    lon = df_for_plotting['lon'],
    lat = df_for_plotting['lat'],
    text = df_for_plotting['text'],
    marker = dict(
        size = 1000*df_for_plotting[str(year)].astype(float)/max_value_years , # size of the dot proportional to the crime rate of the corresponding city, normalized by all-time max
        # sizeref = 2. * max(df_sub['pop']/scale) / (25 ** 2),
        color = "#990000",
        line = dict(width=0.5, color='rgb(40,40,40)'),
        sizemode = 'area'
    ),
   
)
cities.append(city)

layout = dict(
        title = 'Crime rate in major US cities in '+str(year),
       # showlegend = True,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    )

fig = dict(data=cities, layout=layout)


plotly.offline.iplot(fig)



offline.plot(fig, auto_open=True, image = 'png', image_filename="map_us_crime_cities" ,image_width=2000, image_height=1000, 
              filename='/home/staff/julia/at_Northwestern/US_Crime/Figures/map_us_crime_cities.html', validate=True)


'file:///home/staff/julia/at_Northwestern/US_Crime/Figures/map_us_crime_cities.html'

In [ ]:
df_for_plotting['Rank'] = df_for_plotting[str(year)].rank(ascending=False)  # add a new column to rank rows by the value of a given column
df_for_plotting.sort_values(by=[str(year)])

In [ ]:
#ffffff   white-red
#ffe6e6
#ffcccc
#ffb3b3
#ff9999
#ff8080
#ff6666
#ff4d4d
#ff3333
#ff1a1a
#ff0000
#e60000
#cc0000
#b30000
#990000
#800000
#660000

#4d0000     dark-red


In [97]:
###  crime map - CITIES  
### adding now bins for different sizes of circles-crime (calculated with the all-time lows and highs)




year = 2005


### select year:
df_for_plotting = df_crime_cities[[str(year),'lon','lat','City']]
df_for_plotting.dropna(subset=[str(year)], inplace=True)
df_for_plotting['text'] = df_for_plotting['City'] + '<br>Crime rate: ' + (df_for_plotting[str(year)]).astype(str)

# add a new column to rank rows by the value of a given column
df_for_plotting['Rank'] = df_for_plotting[str(year)].rank(ascending=False)  





#### i need the all-time maximum value to rescale/normalize 
#    the areas of the circles consistently across years
lista_all_values = []
max_value_years = 0
i = 1985
while i <=2014:   
    value = df_crime_cities[str(i)].max()
    lista_all_values += df_crime_cities[str(i)].dropna().tolist()  # remove the NANs first or the np.percentile wont work
    if  value > max_value_years:
        max_value_years = value
    i  +=1
list_values_year = df_crime_cities[str(year)].dropna().tolist()  



 
list_pairs_ranks = [[1,9],[9,18],[18,27],[27,36]]     # add an extra rank at the end

colors = ["#4d0000","#ff1a1a","#ff9999","#ffe6e6"]

all_cities = []  # data object for plotting a given year
cont_bines = 0
for pair_values in list_pairs_ranks:   
    #print ("-------",pair_values)
    
    minimo = pair_values[0]
    maximo = pair_values[1]   

    df_select = df_for_plotting[(df_for_plotting['Rank'] >= minimo)  &  (df_for_plotting['Rank'] < maximo)]            
    
   




    city_group = dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = df_select['lon'],
        lat = df_select['lat'],
        text = df_select['text'],
        marker = dict(
            size = 1000*df_select[str(year)].astype(float)/max_value_years , #df_for_plotting[str(year)].astype(float)*10,  # size of the dot proportional to the crime rate of the corresponding city            
            color = colors[cont_bines],
            line = dict(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),
        name = 'ranked {0} - {1}'.format(list_pairs_ranks[cont_bines][0],list_pairs_ranks[cont_bines][1])
    )
    all_cities.append(city_group)
    cont_bines +=1
    
    
layout = dict(
        title = 'Crime rate in major US cities in '+str(year),
        showlegend = True,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    )

fig = dict(data=all_cities, layout=layout)


plotly.offline.iplot(fig)



offline.plot(fig, auto_open=True, image = 'png', image_filename="map_us_crime_cities" ,image_width=2000, image_height=1000, 
              filename='/home/staff/julia/at_Northwestern/US_Crime/Figures/map_us_crime_cities.html', validate=True)


'file:///home/staff/julia/at_Northwestern/US_Crime/Figures/map_us_crime_cities.html'

In [ ]:
## US CITIES crime map + slider (fixed color):



list_years = [1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,\
              1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014]






    
data_slider = []
#### I create a list of data_dicts, one per year that will be displayed with the slider
for year in list_years:
    print (year)
 
    df_for_plotting = df_crime_cities[[str(year),'lon','lat','City']]
    df_for_plotting.dropna(subset=[str(year)], inplace=True)
    df_for_plotting['text'] = df_for_plotting['City'] + '<br>Crime rate: ' + (df_for_plotting[str(year)]).astype(str)

    
    
    data_one_year = dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = df_for_plotting['lon'],
        lat = df_for_plotting['lat'],
        text = df_for_plotting['text'],
        marker = dict(
            size = 1000*df_for_plotting[str(year)].astype(float)/max_value_years , # size of the dot proportional to the crime rate of the corresponding city, normalized by all-time max
            # sizeref = 2. * max(df_sub['pop']/scale) / (25 ** 2),
            color = "#ff3300",
            line = dict(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),

    )
    data_slider.append(data_one_year)
    
    
    

# let's create the steps for the slider
steps = []
for i in range(len(data_slider)):
    step = dict(method='restyle',
                args=['visible', [False] * len(data_slider)],
                label='Year {}'.format(i + 1985))
    step['args'][1][i] = True
    steps.append(step)

sliders = [dict(active=0,
                pad={"t": 1},
                steps=steps)]  



layout = dict(geo=dict(scope='usa',
                       projection={'type': 'albers usa'}),
              sliders=sliders)


fig = dict(data=data_slider, layout=layout)




plotly.offline.iplot(fig)


offline.plot(fig, auto_open=True, image = 'png', image_filename="map_us_crime_cities_slider" ,image_width=2000, image_height=1000, 
              filename='/home/staff/julia/at_Northwestern/US_Crime/Figures/map_us_crime_cities_slider.html', validate=True)


In [ ]:

## US CITIES crime map + slider (fixed color):



list_years = [1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,\
              1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014]






    
data_slider = []
#### I create a list of data_dicts, one per year that will be displayed with the slider
for year in list_years:
    print (year)
 
    df_for_plotting = df_crime_cities[[str(year),'lon','lat','City']]
    df_for_plotting.dropna(subset=[str(year)], inplace=True)
    df_for_plotting['text'] = df_for_plotting['City'] + '<br>Crime rate: ' + (df_for_plotting[str(year)]).astype(str)


    data_one_year = dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = df_for_plotting['lon'],
        lat = df_for_plotting['lat'],
        text = df_for_plotting['text'],
        marker = dict(
            size = 1000*df_for_plotting[str(year)].astype(float)/max_value_years , # size of the dot proportional to the crime rate of the corresponding city, normalized by all-time max
            # sizeref = 2. * max(df_sub['pop']/scale) / (25 ** 2),
            color = "#ff3300",
            line = dict(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),

    )
    
    
    
    
    
    
    data_slider.append(data_one_year)
    
    
    

# let's create the steps for the slider
steps = []
for i in range(len(data_slider)):
    step = dict(method='restyle',
                args=['visible', [False] * len(data_slider)],
                label='Year {}'.format(i + 1985))
    step['args'][1][i] = True
    steps.append(step)

sliders = [dict(active=0,
                pad={"t": 1},
                steps=steps)]  



layout = dict(geo=dict(scope='usa',
                       projection={'type': 'albers usa'}),
              sliders=sliders)


fig = dict(data=data_slider, layout=layout)




plotly.offline.iplot(fig)


offline.plot(fig, auto_open=True, image = 'png', image_filename="map_us_crime_cities_slider" ,image_width=2000, image_height=1000, 
              filename='/home/staff/julia/at_Northwestern/US_Crime/Figures/map_us_crime_cities_slider.html', validate=True)


In [95]:
### US CITY MURDER RATE: MAP + SLIDER




list_years = [1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,\
              1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014]



lista_traces = []  # tot 

trace1_list = []   # one color, all years

trace2_list = []  # another color, all years
trace3_list = []  # another color, all years
trace4_list = []  # another color, all years



#### I create a list of data_dicts, one per year that will be displayed with the slider
for year in list_years:
    print (year)
 
    df_for_plotting = df_crime_cities[[str(year),'lon','lat','City']]
    df_for_plotting.dropna(subset=[str(year)], inplace=True)
    df_for_plotting['text'] = df_for_plotting['City'] + '<br>Crime rate: ' + (df_for_plotting[str(year)]).astype(str)
    df_for_plotting['Rank'] = df_for_plotting[str(year)].rank(ascending=False) ## add column with city ranking for that year
    
    
    ### for top ranked cities:
    minimo = 1
    maximo = 9   

    df_select = df_for_plotting[(df_for_plotting['Rank'] >= minimo)  &  (df_for_plotting['Rank'] < maximo)]            
  
    city_group = dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = df_select['lon'],
        lat = df_select['lat'],
        text = df_select['text'],
        marker = dict(
            size = 1000*df_select[str(year)].astype(float)/max_value_years , #df_for_plotting[str(year)].astype(float)*10,  # size of the dot proportional to the crime rate of the corresponding city            
            color = "#4d0f00",
            line = dict(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),
        name = 'TOP ranked'#'{0} - {1}'.format(list_pairs_ranks[cont_bines][0],list_pairs_ranks[cont_bines][1])
    )
    trace1_list.append(city_group)
    
    
    
    
    
    
    ### for sencond-ranked group of cities:
    minimo = 9
    maximo = 18   

    df_select = df_for_plotting[(df_for_plotting['Rank'] >= minimo)  &  (df_for_plotting['Rank'] < maximo)]            
  
    city_group = dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = df_select['lon'],
        lat = df_select['lat'],
        text = df_select['text'],
        marker = dict(
            size = 1000*df_select[str(year)].astype(float)/max_value_years , #df_for_plotting[str(year)].astype(float)*10,  # size of the dot proportional to the crime rate of the corresponding city            
            color = "#b32400",
            line = dict(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),
        name = '2nd ranked'#'{0} - {1}'.format(list_pairs_ranks[cont_bines][0],list_pairs_ranks[cont_bines][1])
    )
    trace2_list.append(city_group)
    
    
      
        
    ### for sencond-ranked group of cities:
    minimo = 18
    maximo = 27   

    df_select = df_for_plotting[(df_for_plotting['Rank'] >= minimo)  &  (df_for_plotting['Rank'] < maximo)]            
  
    city_group = dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = df_select['lon'],
        lat = df_select['lat'],
        text = df_select['text'],
        marker = dict(
            size = 1000*df_select[str(year)].astype(float)/max_value_years , #df_for_plotting[str(year)].astype(float)*10,  # size of the dot proportional to the crime rate of the corresponding city            
            color = "#ff704d",
            line = dict(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),
        name = '3rd ranked'#'{0} - {1}'.format(list_pairs_ranks[cont_bines][0],list_pairs_ranks[cont_bines][1])
    )
    trace3_list.append(city_group)
    
    
    
        
    ### for sencond-ranked group of cities:
    minimo = 27
    maximo = 35   

    df_select = df_for_plotting[(df_for_plotting['Rank'] >= minimo)  &  (df_for_plotting['Rank'] < maximo)]            
  
    city_group = dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = df_select['lon'],
        lat = df_select['lat'],
        text = df_select['text'],
        marker = dict(
            size = 1000*df_select[str(year)].astype(float)/max_value_years , #df_for_plotting[str(year)].astype(float)*10,  # size of the dot proportional to the crime rate of the corresponding city            
            color = "#ffd6cc",
            line = dict(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),
        name = '4th ranked'#'{0} - {1}'.format(list_pairs_ranks[cont_bines][0],list_pairs_ranks[cont_bines][1])
    )
    trace4_list.append(city_group)
    


lista_traces += trace1_list
lista_traces += trace2_list
lista_traces += trace3_list
lista_traces += trace4_list
num_steps = 30   # as many as years




steps = []
for i in range(num_steps):
    # Start by hiding all traces
    step = dict(
        method = 'restyle',  
        args = ['visible', [False] * len(lista_traces)],
        label='Year {}'.format(i + 1985)
    )
    
    # Enable the N traces we want to see
    step['args'][1][i] = True
    step['args'][1][i+num_steps] = True
    step['args'][1][i+num_steps*2] = True
    step['args'][1][i+num_steps*3] = True
   
    
    # Add step to step list
    steps.append(step)

    
      



sliders = [dict(active=0,
                pad={"t": 1},
                steps=steps)]  



layout = dict(geo=dict(scope='usa',
                       projection={'type': 'albers usa'}),
              sliders=sliders)


fig = dict(data=lista_traces, layout=layout)


# ----------------



 

# # let's create the steps for the slider
# steps = []
# for i in range(len(data_slider)):
#     step = dict(method='restyle',
#                 args=['visible', [False] * len(data_slider)],
#                 label='Year {}'.format(i + 1985))
#     step['args'][1][i] = True
#     steps.append(step)




plotly.offline.iplot(fig)


offline.plot(fig, auto_open=True, image = 'png', image_filename="map_us_crime_cities_slider" ,image_width=2000, image_height=1000, 
              filename='/home/staff/julia/at_Northwestern/US_Crime/Figures/map_us_crime_cities_slider.html', validate=True)


1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014


'file:///home/staff/julia/at_Northwestern/US_Crime/Figures/map_us_crime_cities_slider.html'

In [ ]:
len(list_years)

In [81]:

df_crime_more_cities = pd.read_csv('../Data/Mine/LocalCrimeTrendsInOneVar_murder_rate_vs_time_cities_100000plus_1985_2014.csv')
df_crime_more_cities

,Agency,City,State,1985,1986,1987,1988,1989,1990,1991,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,Birmingham Police Dept,Birmingham,AL,34.4,30.7,29.8,32.5,36.0,47.0,51.6,...,44.3,44.5,37.8,35.9,28.6,NaN,25.3,31.4,29.7,24.5
1,Huntsville Police Dept,Huntsville,AL,7.3,12.5,9.7,9.2,7.5,11.3,NaN,...,13.3,9.5,12.4,10.4,7.3,6.7,7.2,7.6,13.0,8.0
2,Jefferson County Sheriff Department,Jefferson County Sheriff Department,AL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Madison County Sheriff Department,Madison County Sheriff Department,AL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Mobile County Sheriff Department,Mobile County Sheriff Department,AL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Mobile Police Dept,Mobile,AL,12.6,22.6,15.6,15.6,19.1,20.9,20.1,...,14.0,13.6,15.0,16.7,9.7,10.0,11.9,12.7,11.6,12.4
6,Montgomery Police Dept,Montgomery,AL,11.8,11.2,11.2,12.9,17.0,18.2,19.0,...,14.3,13.4,22.8,11.3,15.3,12.1,15.0,21.1,1.5,17.5
7,Anchorage Police Dept,Anchorage,AK,6.1,7.1,6.5,6.0,4.9,4.4,10.7,...,5.8,6.1,7.7,3.6,4.9,4.5,4.0,5.0,4.7,4.0
8,Chandler Police Dept,Chandler,AZ,0.0,5.6,1.4,2.8,1.2,1.1,7.6,...,3.5,3.7,3.6,2.4,2.0,2.5,0.8,1.6,0.8,0.4
9,Gilbert Police Dept,Gilbert,AZ,13.9,11.8,7.4,0.0,0.0,3.4,0.0,...,0.0,1.7,0.5,0.0,1.7,2.4,0.9,2.3,0.4,0.0


In [82]:
### i obtain the lat-long coordinates of the new cities added to the dataframe (not it includes everywhere with 100,000+ people)

from opencage.geocoder import OpenCageGeocode
#from pprint import pprint

key = '3da0043e21ec45fdbf5c098ea9454ff4'  # api key from:  https://opencagedata.com/dashboard  (log in credentials with my google account)
geocoder = OpenCageGeocode(key)



#df['new_col'] = mylist
list_lat = []
list_long = []
for index, row in df_crime_more_cities.iterrows():
    City = row['City']
    State = row['State']
    
    print (City, State)
    query = str(City)+", "+str(State)
                  
    results = geocoder.geocode(query)   

    lat = results[0]['geometry']['lat']
    long = results[0]['geometry']['lng']
    print (lat, long)
    
    list_lat.append(lat)
    list_long.append(long)
    
    
df_crime_more_cities['lat'] = list_lat   
df_crime_more_cities['lon'] = list_long

df_crime_more_cities.to_csv(path_or_buf='../Data/Mine/LocalCrimeTrendsInOneVar_murder_rate_vs_time_cities_100000plus_1985_2014_long_lat.csv', sep=',', na_rep='NA')
print ("written: ../Data/Mine/LocalCrimeTrendsInOneVar_murder_rate_vs_time_cities_100000plus_1985_2014_long_lat.csv")


df_crime_more_cities

,Agency,City,State,1985,1986,1987,1988,1989,1990,1991,...,2007,2008,2009,2010,2011,2012,2013,2014,lat,lon
0,Birmingham Police Dept,Birmingham,AL,34.4,30.7,29.8,32.5,36.0,47.0,51.6,...,37.8,35.9,28.6,NaN,25.3,31.4,29.7,24.5,33.520682,-86.802433
1,Huntsville Police Dept,Huntsville,AL,7.3,12.5,9.7,9.2,7.5,11.3,NaN,...,12.4,10.4,7.3,6.7,7.2,7.6,13.0,8.0,34.729847,-86.585901
2,Jefferson County Sheriff Department,Jefferson County Sheriff Department,AL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.294218,-86.381475
3,Madison County Sheriff Department,Madison County Sheriff Department,AL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.714507,-86.738861
4,Mobile County Sheriff Department,Mobile County Sheriff Department,AL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.690208,-88.104363
5,Mobile Police Dept,Mobile,AL,12.6,22.6,15.6,15.6,19.1,20.9,20.1,...,15.0,16.7,9.7,10.0,11.9,12.7,11.6,12.4,30.694357,-88.043054
6,Montgomery Police Dept,Montgomery,AL,11.8,11.2,11.2,12.9,17.0,18.2,19.0,...,22.8,11.3,15.3,12.1,15.0,21.1,1.5,17.5,32.366966,-86.300648
7,Anchorage Police Dept,Anchorage,AK,6.1,7.1,6.5,6.0,4.9,4.4,10.7,...,7.7,3.6,4.9,4.5,4.0,5.0,4.7,4.0,61.216313,-149.894852
8,Chandler Police Dept,Chandler,AZ,0.0,5.6,1.4,2.8,1.2,1.1,7.6,...,3.6,2.4,2.0,2.5,0.8,1.6,0.8,0.4,33.306713,-111.840849
9,Gilbert Police Dept,Gilbert,AZ,13.9,11.8,7.4,0.0,0.0,3.4,0.0,...,0.5,0.0,1.7,2.4,0.9,2.3,0.4,0.0,33.352826,-111.789024


written: ../Data/Mine/LocalCrimeTrendsInOneVar_murder_rate_vs_time_cities_100000plus_1985_2014_long_lat.csv


In [109]:
###  crime map - CITIES  (fixed color)  - with 400+ cities of 100,000+ people

df_crime_cities = pd.read_csv('../Data/Mine/LocalCrimeTrendsInOneVar_murder_rate_vs_time_cities_100000plus_1985_2014_long_lat.csv')


# i need the all-time maximum value to rescale the city circles accordingly
max_value_years = 0
i = 1985
while i <=2014:   
    value = df_crime_cities[str(i)].max()
    if  value > max_value_years:
        max_value_years = value
    i  +=1





cities = []  # data structure for the plot (list of dict)


year = 1999

df_for_plotting = df_crime_cities[[str(year),'lon','lat','City']]
df_for_plotting.dropna(subset=[str(year)], inplace=True)

df_for_plotting['text'] = df_for_plotting['City'] + '<br>Murder rate: ' + (df_for_plotting[str(year)]).astype(str)


city = dict(
    type = 'scattergeo',
    locationmode = 'USA-states',
    lon = df_for_plotting['lon'],
    lat = df_for_plotting['lat'],
    text = df_for_plotting['text'],
    marker = dict(
        size = 1000*df_for_plotting[str(year)].astype(float)/max_value_years , # size of the dot proportional to the crime rate of the corresponding city, normalized by all-time max
        # sizeref = 2. * max(df_sub['pop']/scale) / (25 ** 2),
        color = "#990000",
        line = dict(width=0.5, color='rgb(40,40,40)'),
        sizemode = 'area'
    ),
   
)
cities.append(city)

layout = dict(
        title = 'Murder rate in major US cities in '+str(year),
       # showlegend = True,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    )

fig = dict(data=cities, layout=layout)


plotly.offline.iplot(fig)



offline.plot(fig, auto_open=True, image = 'png', image_filename="map_us_crime_cities" ,image_width=2000, image_height=1000, 
              filename='/home/staff/julia/at_Northwestern/US_Crime/Figures/map_us_crime_cities.html', validate=True)


'file:///home/staff/julia/at_Northwestern/US_Crime/Figures/map_us_crime_cities.html'

In [110]:

## US CITIES crime map + slider (fixed color):



list_years = [1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,\
              1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014]






    
data_slider = []
#### I create a list of data_dicts, one per year that will be displayed with the slider
for year in list_years:
    print (year)
 
    df_for_plotting = df_crime_cities[[str(year),'lon','lat','City','State']]
    df_for_plotting.dropna(subset=[str(year)], inplace=True)
    df_for_plotting['text'] = df_for_plotting['City']+', ' + df_for_plotting['State'] + '<br>Murder rate: ' + (df_for_plotting[str(year)]).astype(str)


    data_one_year = dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = df_for_plotting['lon'],
        lat = df_for_plotting['lat'],
        text = df_for_plotting['text'],
        marker = dict(
            size = 1000*df_for_plotting[str(year)].astype(float)/max_value_years , # size of the dot proportional to the crime rate of the corresponding city, normalized by all-time max
            # sizeref = 2. * max(df_sub['pop']/scale) / (25 ** 2),
            color = "#990000",
            line = dict(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),

    )
    
    
    
    
    
    
    data_slider.append(data_one_year)
    
    
    

# let's create the steps for the slider
steps = []
for i in range(len(data_slider)):
    step = dict(method='restyle',
                args=['visible', [False] * len(data_slider)],
                label='Year {}'.format(i + 1985))
    step['args'][1][i] = True
    steps.append(step)

sliders = [dict(active=0,
                pad={"t": 1},
                steps=steps)]  


layout = dict(
        title = 'Murder rate in major US cities in '+str(year),
       # showlegend = True,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    sliders=sliders
    )



fig = dict(data=data_slider, layout=layout)




plotly.offline.iplot(fig)


offline.plot(fig, auto_open=True, image = 'png', image_filename="map_us_crime_cities_slider" ,image_width=2000, image_height=1000, 
              filename='/home/staff/julia/at_Northwestern/US_Crime/Figures/map_us_crime_cities_slider.html', validate=True)


1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014


'file:///home/staff/julia/at_Northwestern/US_Crime/Figures/map_us_crime_cities_slider.html'

In [93]:
# from opencage.geocoder import OpenCageGeocode
# #from pprint import pprint

# key = '3da0043e21ec45fdbf5c098ea9454ff4'  # api key from:  https://opencagedata.com/dashboard  (log in credentials with my google account)
# geocoder = OpenCageGeocode(key)

# #results = geocoder.reverse_geocode(44.8303087, -0.5761911)

# query = 'Bijuesca, Spain'   # this part works!!
# results = geocoder.geocode(query)
# print (results[0])

# lat = results[0]['geometry']['lat']
# long = results[0]['geometry']['lng']
# #print (lat, long)


{'annotations': {'DMS': {'lat': "41° 32' 25.83312'' N", 'lng': "1° 55' 13.28232'' W"}, 'MGRS': '30TWL9005499324', 'Maidenhead': 'IN91am99nr', 'Mercator': {'x': -213773.074, 'y': 5064053.763}, 'OSM': {'edit_url': 'https://www.openstreetmap.org/edit?relation=342295#map=17/41.54051/-1.92036', 'url': 'https://www.openstreetmap.org/?mlat=41.54051&mlon=-1.92036#map=17/41.54051/-1.92036'}, 'callingcode': 34, 'currency': {'alternate_symbols': [], 'decimal_mark': ',', 'html_entity': '&#x20AC;', 'iso_code': 'EUR', 'iso_numeric': 978, 'name': 'Euro', 'smallest_denomination': 1, 'subunit': 'Cent', 'subunit_to_unit': 100, 'symbol': '€', 'symbol_first': 1, 'thousands_separator': '.'}, 'flag': '🇪🇸', 'geohash': 'ezqsk61xh2nts1rzhnmu', 'qibla': 106.81, 'sun': {'rise': {'apparent': 1549955220, 'astronomical': 1549949580, 'civil': 1549953480, 'nautical': 1549951500}, 'set': {'apparent': 1549993080, 'astronomical': 1549998660, 'civil': 1549994760, 'nautical': 1549996740}}, 'timezone': {'name': 'Europe/Mad

In [90]:
query = 'Bijuesca, Spain'   
results = geocoder.geocode(query)

lat = results[0]['geometry']['lat']
long = results[0]['geometry']['lng']
print (lat, long)

41.5405092 -1.9203562


In [91]:
print (results[0])

{'annotations': {'DMS': {'lat': "41° 32' 25.83312'' N", 'lng': "1° 55' 13.28232'' W"}, 'MGRS': '30TWL9005499324', 'Maidenhead': 'IN91am99nr', 'Mercator': {'x': -213773.074, 'y': 5064053.763}, 'OSM': {'edit_url': 'https://www.openstreetmap.org/edit?relation=342295#map=17/41.54051/-1.92036', 'url': 'https://www.openstreetmap.org/?mlat=41.54051&mlon=-1.92036#map=17/41.54051/-1.92036'}, 'callingcode': 34, 'currency': {'alternate_symbols': [], 'decimal_mark': ',', 'html_entity': '&#x20AC;', 'iso_code': 'EUR', 'iso_numeric': 978, 'name': 'Euro', 'smallest_denomination': 1, 'subunit': 'Cent', 'subunit_to_unit': 100, 'symbol': '€', 'symbol_first': 1, 'thousands_separator': '.'}, 'flag': '🇪🇸', 'geohash': 'ezqsk61xh2nts1rzhnmu', 'qibla': 106.81, 'sun': {'rise': {'apparent': 1549955220, 'astronomical': 1549949580, 'civil': 1549953480, 'nautical': 1549951500}, 'set': {'apparent': 1549993080, 'astronomical': 1549998660, 'civil': 1549994760, 'nautical': 1549996740}}, 'timezone': {'name': 'Europe/Mad

In [ ]:
# # example slider with multiple traces

# lista_traces = []
# num_steps = 5  # years

# trace1_list = [  # as many (red) traces as steps (years)
#     go.Scatter(y=[1, 2, 3], x=[1, 2, 3], visible=True, line={'color': 'red'}), 
#     go.Scatter(y=[10, 11, 12], x=[1, 2, 3], visible=True, line={'color': 'red'}),   
#     go.Scatter(y=[20, 21, 22], x=[1, 2, 3], visible=True, line={'color': 'red'}),   
#     go.Scatter(y=[30, 31, 32], x=[1, 2, 3], visible=True, line={'color': 'red'}),   
#     go.Scatter(y=[40, 41, 42], x=[1, 2, 3], visible=True, line={'color': 'red'}) 
# ]
# lista_traces += trace1_list


# trace2_list = [
#     go.Scatter(y=[30, 29, 28], x=[1, 2, 3], visible=True, line={'color': 'blue'}),   
#     go.Scatter(y=[10,9, 8], x=[1, 2, 3], visible=True, line={'color': 'blue'}),   
#     go.Scatter(y=[40, 39, 38], x=[1, 2, 3], visible=True, line={'color': 'blue'}),
#     go.Scatter(y=[20, 19, 18], x=[1, 2, 3], visible=True, line={'color': 'blue'}),   
#     go.Scatter(y=[9, 8, 7], x=[1, 2, 3], visible=True, line={'color': 'blue'})  
# ]
# lista_traces += trace2_list




# trace3_list = [
#     go.Scatter(y=[10, 31, 20], x=[1, 2, 3], visible=True, line={'color': 'green'}),   
#     go.Scatter(y=[10, 25, 20], x=[1, 2, 3], visible=True, line={'color': 'green'}),   
#     go.Scatter(y=[10, 20, 21], x=[1, 2, 3], visible=True, line={'color': 'green'}),
#     go.Scatter(y=[10, 15, 20], x=[1, 2, 3], visible=True, line={'color': 'green'}),   
#     go.Scatter(y=[10, 10, 20], x=[1, 2, 3], visible=True, line={'color': 'green'})  
# ]
# lista_traces += trace3_list




# trace4_list = [
#     go.Scatter(y=[15, 31, 25], x=[1, 2, 3], visible=True, line={'color': 'grey'}),   
#     go.Scatter(y=[15, 25, 25], x=[1, 2, 3], visible=True, line={'color': 'grey'}),   
#     go.Scatter(y=[15, 20, 25], x=[1, 2, 3], visible=True, line={'color': 'grey'}),
#     go.Scatter(y=[15, 15, 25], x=[1, 2, 3], visible=True, line={'color': 'grey'}),   
#     go.Scatter(y=[50, 10, 20], x=[1, 2, 3], visible=True, line={'color': 'grey'})  
# ]
# lista_traces += trace4_list




# fig = go.Figure(data=lista_traces)

# steps = []
# for i in range(num_steps):
#     # Hide all traces
#     step = dict(
#         method = 'restyle',  
#         args = ['visible', [False] * len(fig.data)],
#     )
#     # Enable the N traces we want to see
#     step['args'][1][i] = True
#     step['args'][1][i+num_steps] = True
#     step['args'][1][i+num_steps*2] = True
#     step['args'][1][i+num_steps*3] = True
   
    
#     # Add step to step list
#     steps.append(step)

# sliders = [dict(
#     steps = steps,
# )]

# fig.layout.sliders = sliders

# iplot(fig, show_link=False)

In [ ]:
# ## working example map + slider  

# plotly.offline.init_notebook_mode()
# df_2011 = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv')


# ## original dataset is just for one year
# data_slider = [dict(type='choropleth',
#              locations = df_2011['code'].astype(str),
#              z=df_2011['total exports'].astype(float),
#              locationmode='USA-states')]


# # let's create some additional, random data, to fill up different years from the original one-year dataset
# for i in range(5):
#     data_slider.append(data_slider[0].copy())
#     data_slider[-1]['z'] = data_slider[0]['z'] * np.random.rand(*data_slider[0]['z'].shape)

    
    
# # let's create the steps for the slider
# steps = []
# for i in range(len(data_slider)):
#     step = dict(method='restyle',
#                 args=['visible', [False] * len(data_slider)],
#                 label='Year {}'.format(i + 1980))
#     step['args'][1][i] = True
#     steps.append(step)

# sliders = [dict(active=0,
#                 pad={"t": 1},
#                 steps=steps)]    
# layout = dict(geo=dict(scope='usa',
#                        projection={'type': 'albers usa'}),
#               sliders=sliders)

# fig = dict(data=data_slider, 
#            layout=layout)
# plotly.offline.iplot(fig)



# offline.plot(fig, auto_open=True, image = 'png', image_filename="map_slider" ,image_width=2000, image_height=1000, 
#               filename='/home/staff/julia/at_Northwestern/US_Crime/Figures/map_slider.html', validate=True)


In [ ]:
# ### working example sine function with slider:

# ### i build the data list (one per different position that the slider will have)
# data = [dict(
#         visible = False,
#         line=dict(color='#00CED1', width=6),
#         name = '𝜈 = '+str(step),
#         x = np.arange(0,100,0.01),
#         y = np.sin(step*np.arange(0,100,0.01))) for step in np.arange(0,5,0.1)]



# data[1]['visible'] = True



# ### i visualize the plot with slider
# steps = []
# for i in range(len(data)):
#     step = dict(
#         method = 'restyle',  
#         args = ['visible', [False] * len(data)],
#     )
#     step['args'][1][i] = True # Toggle i'th trace to "visible"
#     steps.append(step)

# sliders = [dict(
#     active = 10,
#     currentvalue = {"prefix": "Frequency: "},
#     pad = {"t": 50},
#     steps = steps
# )]

# layout = dict(sliders=sliders)

# fig = dict(data=data, layout=layout)

# #py.iplot(fig, filename='Sine Wave Slider')



# offline.plot(fig, auto_open=True, image = 'png', image_filename="sine_slider" ,image_width=2000, image_height=1000, 
#               filename='/home/staff/julia/at_Northwestern/US_Crime/Figures/sine_slider.html', validate=True)



In [ ]:
# ### example US map by state

# df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv')

# for col in df.columns:
#     df[col] = df[col].astype(str)

# scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
#             [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]

# # df['text'] = df['state'] + '<br>' +\
# #     'Beef '+df['beef']+' Dairy '+df['dairy']+'<br>'+\
# #     'Fruits '+df['total fruits']+' Veggies ' + df['total veggies']+'<br>'+\
# #     'Wheat '+df['wheat']+' Corn '+df['corn']

# data = [ dict(
#         type='choropleth',
#         colorscale = scl,
#         autocolorscale = False,
#         locations = df['code'],
#         z = df['total exports'].astype(float),
#         locationmode = 'USA-states',
#        # text = df['text'],
#         marker = dict(
#             line = dict (
#                 color = 'rgb(255,255,255)',
#                 width = 2
#             ) ),
#         colorbar = dict(
#             title = "Millions USD")
#         ) ]

# layout = dict(
#         title = '2011 US Agriculture Exports by State<br>(Hover for breakdown)',
#         geo = dict(
#             scope='usa',
#             projection=dict( type='albers usa' ),
#             showlakes = True,
#             lakecolor = 'rgb(255, 255, 255)'),
#              )
    
# fig = dict( data=data, layout=layout )




# offline.plot(fig, auto_open=True, image = 'png', image_filename="d3-cloropleth-map" ,image_width=2000, image_height=1000, 
#               filename='/home/staff/julia/at_Northwestern/US_Crime/Figures/d3-cloropleth-map.html', validate=True)

